# 데이터 전처리

## #01. 패키지 참조

In [1]:
import numpy as np
import pandas as pd 

from matplotlib import pyplot as plt
import seaborn as sb

## #02. 데이터 구축

### 1. 데이터 파일 열기

In [2]:
df2018 = pd.read_csv('서울시 우리마을가게 상권분석서비스(상권-추정매출)_2018.csv', encoding='euc-kr')
df2019 = pd.read_csv('서울시 우리마을가게 상권분석서비스(상권-추정매출)_2019.csv', encoding='euc-kr')
df2020 = pd.read_csv('서울시 우리마을가게 상권분석서비스(상권-추정매출)_2020.csv', encoding='euc-kr')
df2021 = pd.read_csv('서울시 우리마을가게 상권분석서비스(상권-추정매출)_2021.csv', encoding='euc-kr')
df상권영역 = pd.read_csv('서울시 우리마을가게 상권분석서비스(상권영역).csv', encoding='euc-kr')
df건축물 = pd.read_csv('서울특별시 건축물대장 법정동 코드정보.csv', encoding='euc-kr')


### 2. 상권영역에 구 이름 병합

#### 상권영역 데이터

서울시에만 해당하는 데이터

In [3]:
df상권영역.head()

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보
0,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN
1,201810,R,전통시장,1001454,봉천제일종합시장,195147,442413,11620,11620595,NaN
2,201810,R,전통시장,1001474,도곡시장,204551,444227,11680,11680650,NaN
3,201810,R,전통시장,1001475,강남개포시장,206065,443310,11680,11680670,NaN
4,201810,R,전통시장,1001412,화곡본동시장,186203,449328,11500,11500590,NaN


#### 법정동 코드정보

전국에 대한 데이터

In [6]:
df건축물['시군구코드'].value_counts()

시군구코드
47780    970
47740    921
47800    875
47810    835
43710    818
        ... 
27145      1
28265      1
25000      1
45118      1
22235      1
Name: count, Length: 500, dtype: int64

### 시군구명, 시군구코드를 추출하여 중복데이터 제거

In [7]:
df시군구 = df건축물.loc[:,['시군구명','시군구코드']]
df시군구 = df시군구.drop_duplicates()
df시군구.dropna(inplace=True)
df시군구

,시군구명,시군구코드
0,영월군,42750
14,유성구,30200
23,평택시,41220
30,용인시 수지구,41465
32,용인시 처인구,41461
...,...,...
82870,대천시,44170
83192,온양시,44190
86860,동구 안심출장소,22145
88397,동해출장소,42105


#### 시군구명과 시군구코드를 상권영역에 결합

In [8]:
df상권영역2 = pd.merge(df상권영역,df시군구,left_on='시군구_코드',right_on='시군구코드',how='left')


In [9]:
df상권영역2['시군구명'].value_counts()

시군구명
강남구     115
관악구      88
동대문구     79
영등포구     74
마포구      71
서초구      70
성북구      69
강서구      69
동작구      65
양천구      63
종로구      63
은평구      61
강동구      60
광진구      60
구로구      59
용산구      58
중구       55
강북구      52
성동구      48
중랑구      48
송파구      45
금천구      41
도봉구      33
서대문구     32
노원구      18
Name: count, dtype: int64

### 파생변수 생성


In [10]:
도심 = ['종로구', '중구', '영등포구', '서초구', '강남구'] 

df상권영역2['도심'] = np.isin(df상권영역2['시군구명'], 도심)

df상권영역2.head()

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보,시군구명,시군구코드,도심
0,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620,False
1,201810,R,전통시장,1001454,봉천제일종합시장,195147,442413,11620,11620595,NaN,관악구,11620,False
2,201810,R,전통시장,1001474,도곡시장,204551,444227,11680,11680650,NaN,강남구,11680,True
3,201810,R,전통시장,1001475,강남개포시장,206065,443310,11680,11680670,NaN,강남구,11680,True
4,201810,R,전통시장,1001412,화곡본동시장,186203,449328,11500,11500590,NaN,강서구,11500,False


In [11]:
df상권영역2.to_excel('상권영역.xlsx', index=False)

### 3. 코로나 전후 매출데이터 
#### 1) 코로나 이전
단순데이터 결합

In [13]:
코로나이전df= pd.concat([df2018,df2019])
코로나이전df.reset_index(drop=True,inplace=True)
len(코로나이전df)

298244

골목상권 데이터만 추출


In [14]:
코로나이전df2 = 코로나이전df.loc[코로나이전df['상권_구분_코드_명']=='골목상권',:]
코로나이전df2.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
2,2018,1,A,골목상권,1000289,인촌로17가길,CS300021,문구,6044732,23,...,0,23,0,0,0,0,8,7,8,4
3,2018,1,A,골목상권,1000515,남부순환로70길,CS200030,피부관리실,11863957,40,...,0,0,36,0,0,0,13,23,0,1
4,2018,1,A,골목상권,1000902,논현로63길,CS200029,네일숍,1766586,19,...,3,0,15,0,0,11,0,0,5,2
5,2018,1,A,골목상권,1000407,연서로33길,CS200007,치과의원,140020113,2150,...,0,1033,1117,32,280,324,402,458,654,1
6,2018,1,A,골목상권,1000407,연서로33길,CS200008,한의원,145038784,6360,...,20,2712,3648,10,391,896,1326,1930,1807,1


코로나 이전과 이후의 조사범위를 동일하게 설정
코로나19 발생시점인 2020년 1월을 기준으로 2018년 1,2,3,4분기, 2019년 1,2,3,4분기 데이터와 2020년 1,2,3,4분기, 2021년 1,2분기 데이터만 수집된 상태.

그러므로 코로나 이전 데이터에서 2018년 1,2분기를 분석 범위에서 제외해야 코로나 전후에 대한 데이터 분포가 균형을 이루게 된다.

In [15]:
제외할index = 코로나이전df2.query("(기준_년_코드==2018 and (기준_분기_코드==1 or 기준_분기_코드==2))").index
코로나이전df3 = 코로나이전df2.drop(제외할index)
코로나이전df3.reset_index(drop=True, inplace=True)
#코로나이전df3.to_excel('코로나이전_매출데이터.xlsx', index=False)
코로나이전df3

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2018,3,A,골목상권,1000094,마장로39길,CS200001,일반교습학원,182447499,644,...,25,291,308,0,5,39,464,92,0,5
1,2018,3,A,골목상권,1000066,원효로41길,CS100009,호프-간이주점,29160217,889,...,440,580,156,31,55,74,216,331,29,4
2,2018,3,A,골목상권,1000061,신흥로20길,CS300004,핸드폰,155314528,1128,...,0,627,448,0,197,56,416,189,216,1
3,2018,3,A,골목상권,1000878,주흥길,CS200029,네일숍,6170041,100,...,14,15,85,6,28,27,15,15,9,3
4,2018,3,A,골목상권,1000853,동광로12길,CS300033,철물점,68405170,569,...,0,276,113,0,25,59,99,147,59,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139603,2019,4,A,골목상권,1000440,세검정로1길,CS300014,신발,21965900,887,...,0,366,521,0,20,79,113,211,464,3
139604,2019,4,A,골목상권,1000910,도산대로81길,CS300022,화장품,1724533532,1824,...,382,782,896,17,377,630,293,218,144,7
139605,2019,4,A,골목상권,1000181,망우로21길,CS100004,양식음식점,562291801,23923,...,228,12935,9718,469,6166,6432,4792,3474,1319,2
139606,2019,4,A,골목상권,1000441,세무서2길,CS300002,편의점,518371562,81848,...,17654,57240,24608,2813,30887,15345,14338,12869,5596,2


### 코로나 이후 매출 데이터


In [16]:

코로나이후df = pd.concat([df2020,df2021])
코로나이후df.reset_index(drop=True,inplace=True)

코로나이후df2 = 코로나이후df.loc[코로나이후df['상권_구분_코드_명']=='골목상권',:]
코로나이후df2.to_excel("코로나이후_매출데이터.xlsx", index=False)
코로나이후df2


,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
2,2020,1,A,골목상권,1000022,창신2길,CS200001,일반교습학원,22367209.0,82,...,0,54,28,0,0,0,82,0,0,2
4,2020,1,A,골목상권,1000001,계동길,CS100001,한식음식점,431442455.0,18548,...,345,8752,7776,611,3828,4356,3428,2614,1690,19
5,2020,3,A,골목상권,1000127,광나루로12길,CS200016,당구장,424034.0,17,...,9,17,0,0,0,17,0,0,0,1
8,2020,1,A,골목상권,1000001,계동길,CS100002,중식음식점,25767161.0,1413,...,0,592,694,145,368,240,368,96,69,1
9,2020,1,A,골목상권,1000001,계동길,CS100003,일식음식점,55484866.0,1272,...,241,622,508,0,134,245,275,314,162,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197054,2021,1,A,골목상권,1000001,계동길,CS100005,제과점,165688274.0,18512,...,676,7523,9187,208,2471,4577,3857,2990,2609,4
197055,2021,1,A,골목상권,1000001,계동길,CS100004,양식음식점,199448496.0,4442,...,66,1778,2175,31,1505,1155,439,566,256,3
197056,2021,1,A,골목상권,1000001,계동길,CS100003,일식음식점,60325474.0,1563,...,136,519,635,0,185,339,242,279,112,2
197057,2021,1,A,골목상권,1000001,계동길,CS100002,중식음식점,39791852.0,2042,...,11,825,1041,243,629,307,322,269,97,1


#### 코로나 이전과 이후 데이터를 결합

코로나 이전 데이터에 명목형 변수 추가


In [17]:
코로나이전df3['covid19']=0
코로나이후df2['covid19']=1



C:\Users\Hamin\AppData\Local\Temp\ipykernel_12592\629423170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  코로나이후df2['covid19']=1


In [18]:
매출데이터_최종 = pd.concat([코로나이전df3,코로나이후df2])

매출데이터_최종.reset_index(drop=True,inplace=True)

매출데이터_최종.head()


,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수,covid19
0,2018,3,A,골목상권,1000094,마장로39길,CS200001,일반교습학원,182447499.0,644,...,291,308,0,5,39,464,92,0,5,0
1,2018,3,A,골목상권,1000066,원효로41길,CS100009,호프-간이주점,29160217.0,889,...,580,156,31,55,74,216,331,29,4,0
2,2018,3,A,골목상권,1000061,신흥로20길,CS300004,핸드폰,155314528.0,1128,...,627,448,0,197,56,416,189,216,1,0
3,2018,3,A,골목상권,1000878,주흥길,CS200029,네일숍,6170041.0,100,...,15,85,6,28,27,15,15,9,3,0
4,2018,3,A,골목상권,1000853,동광로12길,CS300033,철물점,68405170.0,569,...,276,113,0,25,59,99,147,59,3,0


#### 분석에 사용할 요인만 추출 + 파생변수 생성

In [19]:
분석데이터 = 매출데이터_최종.loc[:,['기준_년_코드', '기준_분기_코드', '상권_코드', '분기당_매출_금액', '주중_매출_금액', '주말_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액', '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', '점포수', 'covid19']]

분석데이터['일과시간_금액'] = 분석데이터['시간대_06~11_매출_금액'] + 분석데이터['시간대_11~14_매출_금액'] + 분석데이터['시간대_14~17_매출_금액']

분석데이터['일과이후_금액'] = 분석데이터['시간대_17~21_매출_금액'] + 분석데이터['시간대_21~24_매출_금액'] + 분석데이터['시간대_00~06_매출_금액']

분석데이터['청년금액'] = (분석데이터['연령대_10_매출_금액'] + 분석데이터['연령대_20_매출_금액'] + 분석데이터['연령대_30_매출_금액'])
분석데이터['중년금액'] = (분석데이터['연령대_40_매출_금액'] + 분석데이터['연령대_50_매출_금액'])
분석데이터['장년금액'] = (분석데이터['연령대_60_이상_매출_금액'])
분석데이터.drop(['시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액'], axis=1, inplace=True)

분석데이터.to_excel('분석데이터.xlsx', index=False)


print(분석데이터.shape)

print(분석데이터.info())

분석데이터.head()

(265928, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265928 entries, 0 to 265927
Data columns (total 15 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   기준_년_코드    265928 non-null  int64  
 1   기준_분기_코드   265928 non-null  int64  
 2   상권_코드      265928 non-null  int64  
 3   분기당_매출_금액  265928 non-null  float64
 4   주중_매출_금액   265928 non-null  float64
 5   주말_매출_금액   265928 non-null  float64
 6   남성_매출_금액   265928 non-null  float64
 7   여성_매출_금액   265928 non-null  float64
 8   점포수        265928 non-null  int64  
 9   covid19    265928 non-null  int64  
 10  일과시간_금액    265928 non-null  float64
 11  일과이후_금액    265928 non-null  float64
 12  청년금액       265928 non-null  int64  
 13  중년금액       265928 non-null  float64
 14  장년금액       265928 non-null  int64  
dtypes: float64(8), int64(7)
memory usage: 30.4 MB
None


,기준_년_코드,기준_분기_코드,상권_코드,분기당_매출_금액,주중_매출_금액,주말_매출_금액,남성_매출_금액,여성_매출_금액,점포수,covid19,일과시간_금액,일과이후_금액,청년금액,중년금액,장년금액
0,2018,3,1000094,182447499.0,181667733.0,779766.0,81108758.0,83644304.0,5,0,66151380.0,116296119.0,10902631,153850431.0,0
1,2018,3,1000066,29160217.0,19442827.0,9717390.0,18910409.0,3478669.0,4,0,4182259.0,24977958.0,3923691,16685126.0,1780259
2,2018,3,1000061,155314528.0,152130813.0,3183715.0,117007487.0,9044514.0,1,0,106593580.0,48720948.0,37567976,10158739.0,78325286
3,2018,3,1000878,6170041.0,5705645.0,464396.0,570521.0,5599520.0,3,0,2473589.0,3696452.0,3605346,2157426.0,407269
4,2018,3,1000853,68405170.0,58206064.0,10199106.0,25402838.0,19718192.0,3,0,50288107.0,18117063.0,6488172,30182571.0,8450288
